In [1]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random
import requests as req
import urllib
from time import sleep

In [2]:
#Load in API key
with open('owm_api.txt', 'r') as file_object:
    #Text document that is untracked in project-one directory (NOT pushed to git) with API key
    #Create a file called untracked_api_key.txt with your key and DO NOT add to git (add to .gitignore or have outside repository)
    api_key  = str(file_object.readline()).strip()

#Openweather map API base url
base_url = "http://api.openweathermap.org/data/2.5/weather"

In [3]:


#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 5):
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Pass query to OpenWeatherMap API to check if city is in database
    
    query_city = cityToCheck
    query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
    response = req.get(query_url)
    json_data = response.json()
    
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    #While also not adding the specified city if not in OWM database
    if cityToCheck not in cityList and json_data['cod'] != '404':
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
    print("uniqueCityCount " + str(uniqueCityCount))
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Creation of column for city number
raw_cities_df['City Number'] = raw_cities_df.index + 1
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

uniqueCityCount 1
uniqueCityCount 2
uniqueCityCount 3
uniqueCityCount 4
uniqueCityCount 5


In [4]:
#Openweather map API base url
base_url = "http://api.openweathermap.org/data/2.5/weather"

In [5]:
raw_cities_df.head(25)
#print(raw_cities_df['City Number'])

,City Number,City Name
0,1,punta arenas
1,2,mehamn
2,3,bambous virieux
3,4,nikolskoye
4,5,puerto ayora


In [6]:
#Declaration of list to hold city data for list => dataframe
city_data_list = []

In [16]:
#OWM API settings
settings = {"units": "imperial","appid": api_key}

#Print parameters being requested from owm API before retrival
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")
views = {"summary":["name", "sys.country","coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]}
summary = ["name", "sys.country", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'sys.country', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']


In [17]:
#Get current weather for each city in cityList
cityCount = 1
for city in raw_cities_df["City Name"]:
    sleep(1.2)
    try:
        print("Processing city " + str(cityCount)+ " of " +str(len(raw_cities_df["City Name"])) + ", "+ city +":") 
        current_city= ow.get_current(city, **settings)
        
        sleep(1.2)
        #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
        summary_output = current_city(*summary)
        print(str(summary_output))
        #print(current_city["sys"]["country"])
        city_url = base_url + "&q=" + urllib.request.pathname2url(city)
        print (city_url)
        city_data_list.append(current_city(*views["summary"]))

    except Exception as ex:
        print(ex)
        cityCount = cityCount - 1
    cityCount = cityCount + 1


Processing city 1 of 5, punta arenas:
('Punta Arenas', 'CL', -53.16, -70.91, 43.15, 100, 29.1, 36)
http://api.openweathermap.org/data/2.5/weather&q=punta%20arenas
Processing city 2 of 5, mehamn:
('Mehamn', 'NO', 71.03, 27.85, 30.52, 92, 8.05, 90)
http://api.openweathermap.org/data/2.5/weather&q=mehamn
Processing city 3 of 5, bambous virieux:
('Bambous Virieux', 'MU', -20.34, 57.76, 75.2, 94, 4.7, 40)
http://api.openweathermap.org/data/2.5/weather&q=bambous%20virieux
Processing city 4 of 5, nikolskoye:
('Nikolskoye', 'RU', 59.7, 30.79, 26.6, 86, 2.24, 75)
http://api.openweathermap.org/data/2.5/weather&q=nikolskoye
Processing city 5 of 5, puerto ayora:
('Puerto Ayora', 'EC', -0.74, -90.35, 84.2, 66, 10.29, 20)
http://api.openweathermap.org/data/2.5/weather&q=puerto%20ayora


In [18]:
print(city_data_list)

[('Punta Arenas', 'CL', -53.16, -70.91, 43.15, 100, 29.1, 36), ('Mehamn', 'NO', 71.03, 27.85, 30.52, 92, 8.05, 90), ('Bambous Virieux', 'MU', -20.34, 57.76, 75.2, 94, 4.7, 40), ('Nikolskoye', 'RU', 59.7, 30.79, 26.6, 86, 2.24, 75), ('Puerto Ayora', 'EC', -0.74, -90.35, 84.2, 66, 10.29, 20)]


In [19]:
city_data_df = pd.DataFrame(city_data_list)
#Set column headers to summary/view labels
city_data_df.columns = summary
city_data_df.head()


,name,sys.country,coord.lat,coord.lon,main.temp,main.humidity,wind.speed,clouds.all
0,Punta Arenas,CL,-53.16,-70.91,43.15,100,29.10,36
1,Mehamn,NO,71.03,27.85,30.52,92,8.05,90
2,Bambous Virieux,MU,-20.34,57.76,75.20,94,4.70,40
3,Nikolskoye,RU,59.70,30.79,26.60,86,2.24,75
4,Puerto Ayora,EC,-0.74,-90.35,84.20,66,10.29,20


In [20]:
city_data_df.to_csv("city_weather_data_scraped.csv", mode ='w')

In [ ]:
#Documentation on OWM wrapper ---> https://github.com/crazycapivara/openweathermapy
#openweathermap complete city list:
    #http://openweathermap.org/help/city_list.txt 
        #one method ---> check against list
        #another method ---> skip if error returned 
    
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
#Mapping ----> http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.Series.map.html
#https://stackoverflow.com/questions/26666919/python-pandas-add-column-in-dataframe-from-list
#https://stackoverflow.com/questions/42632470/how-to-add-dictionaries-to-a-dataframe-as-a-row

#Put requested data into a dictionary that contains a list (intention to append to dataframe)
    #views = {"summary":["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]}
    #print(current_city(*views["summary"]))
    #city_data_list.append(current_city(*views["summary"]))

